In [2]:
import json
import os
from glob import glob
import numpy as np
import pandas as pd
from collections import Counter
import copy
import csv

dir_val = "/home/data/kbh/DCASE_eval/ER616_Real/"
submissions_dir = "/home/data/kbh/DCASE_eval/test_output"
list_out = glob(os.path.join("/home/data/kbh/DCASE_eval/",dir_val,"*.npy"))
#print(aa)
i = 0

os.makedirs(submissions_dir,exist_ok=True)

for path in list_out:
    a = np.load(path) # 900 X 3 X 13
    
    name_target = path.split('/')[-1]
    id_target = name_target.split('.')[0]
    
    frame_mean = np.mean(a, axis=0) # 3 X 13
    point_mean = np.mean(frame_mean, axis = 0) # 13
    
    #실제 진행되는 연산
    
    pred_sed_max= a.max(axis=-1) # 900, 3 => 실제 max 값
    pred_sed_max_idx = a.argmax(axis=-1) #900, 3 => max index 요거 두개로 쓰는거, 단 output이 있다!
    pred_sed = np.zeros_like(a) # 900 X 3 X 13
    frame_length = [857, 381, 1183, 1042, 1654, 2537, 1747, 1522, 1101, 1417, 3117, 2142, 848, 1392, 1281, 987, 1336, 948, 2015, 2992, 850, 690, 618, 928, 1029, 2265, 1474, 616, 869, 2181, 800, 714, 971, 1076, 837, 597, 2089, 997, 986, 3299, 856, 729, 1154, 581, 1108, 2617, 2713, 2155, 617, 822, 662, 736]
    T = frame_length[i]
    for t_idx in range(T): #T => 실제 frame 길이 - length.txt 참조
        #for track_idx in range(2):
        if t_idx < pred_sed.shape[0]:
            for track_idx in range(3):
                pred_sed[t_idx, track_idx, pred_sed_max_idx[t_idx, track_idx]] = pred_sed_max[t_idx, track_idx]
        else:
            print("error : ", id_target, t_idx, pred_sed.shape[0])
    #print(a.shape, pred_sed_max.shape, pred_sed_max_idx.shape) 
    
    
    
    frame_mean2 = np.mean(pred_sed_max, axis = 0)
    #print(point_mean, point_mean.shape, frame_mean2, frame_mean2.shape)
    threshold_list = [0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5]
    for j in range(13):
        pred_sed[:, :,j] = a[:, :,j] > threshold_list[j]
    #print(pred_sed)
    i += 1
    # 0.5
    #print(point_mean)
    
    pred_sed = pred_sed.astype(np.float32)
    azi = np.arctan2(pred_doa[..., 1], pred_doa[..., 0])
    elev = np.arctan2(pred_doa[..., 2], np.sqrt(pred_doa[..., 0]**2 + pred_doa[..., 1]**2))
    pred_doa = np.stack((azi, elev), axis=-1) # (N, T, tracks, (azi, elev))
    #pdb.set_trace()
    pred_sed_f = np.array([pred_sed]) #1400, 5, 13
    pred_doa_f = np.array([pred_doa]) #1400, 5, 2 꼴이여야 함
    fn = id_target
    video_len_list = [pred_sed.shape[0]]
    pred_dcase_format_dict = to_dcase_format(pred_sed_f, pred_doa_f, video_len_list)
    csv_path = submissions_dir.joinpath(fn + '.csv')
    print(csv_path)
    #pdb.set_trace()
    for key in pred_dcase_format_dict[0]:
        for value in pred_dcase_format_dict[0][key]:
            with csv_path.open('a') as f:
                f.write('{},{},{},{}\n'.format(key, value[0], value[1], value[2]))
    #Post processing : value는 [[1, -17, -24]]식, 즉, key+1, value[0], value[1], value[2]까지 쓰면 됨
    print('Results are saved to {}\n'.format(str(submissions_dir)))    

error :  mix052_pred_sed 800 800
error :  mix052_pred_sed 801 800
error :  mix052_pred_sed 802 800
error :  mix052_pred_sed 803 800
error :  mix052_pred_sed 804 800
error :  mix052_pred_sed 805 800
error :  mix052_pred_sed 806 800
error :  mix052_pred_sed 807 800
error :  mix052_pred_sed 808 800
error :  mix052_pred_sed 809 800
error :  mix052_pred_sed 810 800
error :  mix052_pred_sed 811 800
error :  mix052_pred_sed 812 800
error :  mix052_pred_sed 813 800
error :  mix052_pred_sed 814 800
error :  mix052_pred_sed 815 800
error :  mix052_pred_sed 816 800
error :  mix052_pred_sed 817 800
error :  mix052_pred_sed 818 800
error :  mix052_pred_sed 819 800
error :  mix052_pred_sed 820 800
error :  mix052_pred_sed 821 800
error :  mix052_pred_sed 822 800
error :  mix052_pred_sed 823 800
error :  mix052_pred_sed 824 800
error :  mix052_pred_sed 825 800
error :  mix052_pred_sed 826 800
error :  mix052_pred_sed 827 800
error :  mix052_pred_sed 828 800
error :  mix052_pred_sed 829 800
error :  m

NameError: name 'pred_doa' is not defined